## Read in the data

In [67]:
# Import libraries
import pandas as pd, random as rd, numpy as np
import math
from IPython.display import clear_output, Image, display

In [68]:
#Skip the header row
qual_fname = '../final_data/cleaned_qual_Final.csv'
cleaned_qual = pd.read_csv(qual_fname, sep=',', encoding = "ISO-8859-1")

In [69]:
# cleaned_qual = cleaned_qual[2:]

In [70]:
len(cleaned_qual.username.unique())

701

In [71]:
cleaned_qual.groupby("condition_group")["username"].count()

condition_group
0    100
1    103
2     98
3    101
4    100
5    100
6     99
Name: username, dtype: int64

In [72]:
temp = cleaned_qual.loc[cleaned_qual.condition_group == 1,"username"] # Grab condition group == 1 subset of data
drop_index = np.random.choice(temp.index, 1, replace=False)[0] # Select the index of a username to drop

In [73]:
print("Dropped username was: {}".format(cleaned_qual.iloc[drop_index]["username"]))
# Dropped username1412

Dropped username was: username1412


In [74]:
cleaned_qual = cleaned_qual.drop(drop_index) # Drop the selected username

In [75]:
len(cleaned_qual) # Confirm length

700

## Cybersecurity Quiz Recoding

In [76]:
# Calculate scores for cybersecurity questions individually
cleaned_qual['url_result'] = np.where(cleaned_qual['CK_URL']=='That information entered into the site is encrypted',1,0)
cleaned_qual['phishing_result'] = np.where((cleaned_qual['CK_phishing']=='All of the above') |
                                           (cleaned_qual['CK_phishing']=='Sending someone an email that contains a malicious link that is disguised to look like an email from someone the person knows,Creating a fake website that looks nearly identical to a real website, in order to trick users into entering their login information,Sending someone a text message that contains a malicious link that is disguised to look like  a notification that the person has won a contest,All of the above') |  
                                           (cleaned_qual['CK_phishing']=='Sending someone an email that contains a malicious link that is disguised to look like an email from someone the person knows,Creating a fake website that looks nearly identical to a real website, in order to trick users into entering their login information,Sending someone a text message that contains a malicious link that is disguised to look like  a notification that the person has won a contest'),1,0)
cleaned_qual['botnet_result'] = np.where(cleaned_qual['CK_botnet']=='Botnet',1,0)
cleaned_qual['wifi_result'] = np.where(cleaned_qual['CK_wifi_encryption']=='False',1,0)
cleaned_qual['twostep_result'] = np.where(cleaned_qual['CK_two_step']=='Click to select Picture 1',1,0)
cleaned_qual['password_result'] = np.where(cleaned_qual['CK_password']=='WTh!5Z',1,0)
cleaned_qual['ransomware_result'] = np.where(cleaned_qual['CK_ransomware']=='Ransomware',1,0)
cleaned_qual['browsing_result'] = np.where(cleaned_qual['CK_browsing']=='Yes',1,0)
cleaned_qual['gps_result'] = np.where(cleaned_qual['CK_GPS']=='False',1,0)
cleaned_qual['email_result'] = np.where(cleaned_qual['CK_email_ecryption']=='False',1,0)
cleaned_qual['report_result'] = np.where(cleaned_qual['CK_report']=='One',1,0)
cleaned_qual['safety_result'] = np.where(cleaned_qual['CK_wifi_safety']=='No, it is not safe',1,0)
cleaned_qual['vpn_result'] = np.where(cleaned_qual['CK_VPN']=='Use of insecure wi-fi networks',1,0)

In [77]:
#Now calculate the aggregated score of cybersecurity quiz
#Get the index of first and last column, sum up the scores of all columns between
cleaned_qual['cyber_quiz_score']= cleaned_qual.iloc[:, cleaned_qual.columns.get_loc('url_result'):(cleaned_qual.columns.get_loc('vpn_result')+1)].sum(axis=1)

In [78]:
#Check cybersecurity scores
#cleaned_qual['cyber_quiz_score']

## Previous Experience Recoding

In [79]:
#Next we move to the calculation of previous phishing experience
#Recode yes or no to 1/0, keep NA values
cleaned_qual = cleaned_qual.replace({
                'Yes': 1,
                'No': 0,
                'Prefer not to answer': 0
            })
#Add up eight items to get an aggregated score
cleaned_qual['PE_score']= cleaned_qual.iloc[:, cleaned_qual.columns.get_loc('PE_1'):(cleaned_qual.columns.get_loc('PE_8')+1)].sum(axis=1)

In [80]:
#Check the previous experience score
#cleaned_qual['PE_score']

## Brand Usage Recoding

In [81]:
#Now moving to the brand usage questions
#First rename the columns to make the headers more descriptive
cleaned_qual = cleaned_qual.rename(columns={'Q186': 'venmo_freq', 'Q119': 'chase_freq', 'Q120': 'yahoo_freq'})
#Recode values
cleaned_qual = cleaned_qual.replace({
                'Less than once a month': 1,
                'Once a month': 2,
                'Several times a month': 3,
                'Once a week': 4,
                'Several times a week': 5,
                'Daily': 6,
                "I'm not sure": 0
            })
#Fill NA values with 0
cleaned_qual.fillna({'venmo_freq':0, 'chase_freq':0, 'yahoo_freq':0}, inplace=True)

In [82]:
#Recalculate the brand usage score, using the original value minus mean
cleaned_qual['venmo_normalized'] = cleaned_qual['venmo_freq'] - cleaned_qual['venmo_freq'].mean()
cleaned_qual['chase_normalized'] = cleaned_qual['chase_freq'] - cleaned_qual['chase_freq'].mean()
cleaned_qual['yahoo_normalized'] = cleaned_qual['yahoo_freq'] - cleaned_qual['yahoo_freq'].mean()

In [83]:
# #Compute the mean value of brand usage score for those who use any one of the three brands
# venmo_mean = cleaned_qual.loc[cleaned_qual['venmo_freq'] > 0]['venmo_freq'].mean()
# chase_mean = cleaned_qual.loc[cleaned_qual['chase_freq'] > 0]['chase_freq'].mean()
# yahoo_mean = cleaned_qual.loc[cleaned_qual['yahoo_freq'] > 0]['yahoo_freq'].mean()
# #Create three new columns to record the final score
# cleaned_qual['venmo_final'] = cleaned_qual['venmo_freq']
# cleaned_qual['chase_final'] = cleaned_qual['chase_freq']
# cleaned_qual['yahoo_final'] = cleaned_qual['yahoo_freq']
# #For users of the brand, their final score of brand usage would be the original value minus the mean value
# cleaned_qual.loc[cleaned_qual['venmo_freq'] > 0,'venmo_final'] = cleaned_qual['venmo_freq'] - venmo_mean
# cleaned_qual.loc[cleaned_qual['chase_freq'] > 0,'chase_final'] = cleaned_qual['chase_freq'] - chase_mean
# cleaned_qual.loc[cleaned_qual['yahoo_freq'] > 0,'yahoo_final'] = cleaned_qual['yahoo_freq'] - yahoo_mean

In [84]:
# cleaned_qual['chase_final']

# Select certain columns of survey data to merge

In [85]:
#Print out all column names, so we know which ones to select
cleaned_qual.columns.values.tolist()

['10_EQ-title',
 '10_Q51',
 '10_Q51_6_TEXT',
 '10_Q52',
 '10_Q56_Click Count',
 '10_Q56_First Click',
 '10_Q56_Last Click',
 '10_Q56_Page Submit',
 '1_EQ-title',
 '1_Q51',
 '1_Q51_6_TEXT',
 '1_Q52',
 '1_Q56_Click Count',
 '1_Q56_First Click',
 '1_Q56_Last Click',
 '1_Q56_Page Submit',
 '2_EQ-title',
 '2_Q51',
 '2_Q51_6_TEXT',
 '2_Q52',
 '2_Q56_Click Count',
 '2_Q56_First Click',
 '2_Q56_Last Click',
 '2_Q56_Page Submit',
 '3_EQ-title',
 '3_Q51',
 '3_Q51_6_TEXT',
 '3_Q52',
 '3_Q56_Click Count',
 '3_Q56_First Click',
 '3_Q56_Last Click',
 '3_Q56_Page Submit',
 '4_EQ-title',
 '4_EQ-title - Sentiment',
 '4_EQ-title - Sentiment Polarity',
 '4_EQ-title - Sentiment Score',
 '4_EQ-title - Topics',
 '4_Q51',
 '4_Q51_6_TEXT',
 '4_Q52',
 '4_Q56_Click Count',
 '4_Q56_First Click',
 '4_Q56_Last Click',
 '4_Q56_Page Submit',
 '5_EQ-title',
 '5_Q51',
 '5_Q51_6_TEXT',
 '5_Q52',
 '5_Q56_Click Count',
 '5_Q56_First Click',
 '5_Q56_Last Click',
 '5_Q56_Page Submit',
 '6_EQ-title',
 '6_Q51',
 '6_Q51_6_TEX

In [86]:
#Keep username and MID as participant ID, cybersecurity quiz score, previous experience score, recalculated brand usage scores
#Also include demographic information (gender, age, education, occupation)
cleaned_qual_selected = cleaned_qual[['MID','username','cyber_quiz_score','PE_score','venmo_freq','chase_freq','yahoo_freq','venmo_normalized','chase_normalized','yahoo_normalized','DQ_Gender','DQ_Age','DQ_Educ','DQ_Occ']]

# Join the dataframe with log data

In [87]:
#Write the aggregated survey data into a csv file
#cleaned_qual_selected.to_csv('qual_agg_data.csv', sep=',', index=False)

In [88]:
#Read the log summary file
log_fname_master = '../final_data/cleaned_agg_log_Final.csv'
cleaned_log_master = pd.read_csv(log_fname_master, sep=',')
log_fname_adjusted = '../final_data/cleaned_agg_log_link_adjusted_final.csv'
cleaned_log_adjusted = pd.read_csv(log_fname_adjusted, sep=',')

In [89]:
#Merge the two dataframes together
cleaned_merge_master = pd.merge(cleaned_qual_selected, cleaned_log_master, on='username')
cleaned_merge_adjusted = pd.merge(cleaned_qual_selected, cleaned_log_adjusted, on='username')

# Calculate Brand Usage Score

In [90]:
#Create a function that operates on the value of "email_sender"
def f(row):
    if row['email_sender'] == "chase":
        val = row['chase_normalized']
    elif row['email_sender'] == "venmo":
        val = row['venmo_normalized']
    elif row['email_sender'] == "yahoo":
        val = row['yahoo_normalized']
    else:
        val = 0
    return val

In [91]:
#Apply it to the dataframe's new column "brand_usage"
cleaned_merge_master['brand_usage'] = cleaned_merge_master.apply(f, axis=1)
cleaned_merge_adjusted['brand_usage'] = cleaned_merge_adjusted.apply(f, axis=1)

In [92]:
#Write the joint dataframe into a csv file
cleaned_merge_master.to_csv('data_master.csv', sep=',', index=False)
cleaned_merge_adjusted.to_csv('data_adjusted.csv', sep=',', index=False)

In [93]:
len(cleaned_merge_master.username.unique())

700

In [94]:
len(cleaned_merge_adjusted.username.unique())

700